<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/E2Moon_space_travel_deepsekv3dot1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

import os
import sqlite3
from openai import OpenAI
from typing import List, Dict, Any
from datetime import datetime
import logging
import time
import re

# Configure logging
logging.basicConfig(
    filename='mission_planning.log',
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Get the API key
try:
    from google.colab import userdata
    api_key = userdata.get('DEEPSEEK_API_KEY')
except ImportError:
    api_key = os.getenv('DEEPSEEK_API_KEY')
if not api_key:
    logger.error("DEEPSEEK_API_KEY not found.")
    raise ValueError("DEEPSEEK_API_KEY not found.")

# Initialize the OpenAI client for DeepSeek API
client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1"
)

# --- Tools ---
def generate_text(prompt: str, system_prompt: str = None, model: str = "deepseek-reasoner", temperature: float = 0.7, max_tokens: int = 2048, retries: int = 3) -> str:
    """Generates a response from the LLM with retry logic."""
    messages = [{"role": "user", "content": prompt}]
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})

    attempt = 0
    current_temperature = temperature

    while attempt < retries:
        try:
            logger.debug(f"Attempt {attempt + 1}/{retries} to generate text with prompt: {prompt[:100]}...")
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                max_tokens=max_tokens,
                n=1,
                stop=None,
                temperature=current_temperature
            )
            result = response.choices[0].message.content.strip()
            if not result:
                logger.warning(f"Empty response from LLM on attempt {attempt + 1}.")
                attempt += 1
                current_temperature = min(current_temperature + 0.2, 1.0)
                time.sleep(1)
                continue
            logger.info(f"Generated text for prompt: {prompt[:100]}...")
            return result
        except Exception as e:
            logger.error(f"LLM error on attempt {attempt + 1}: {str(e)}")
            attempt += 1
            current_temperature = min(current_temperature + 0.2, 1.0)
            time.sleep(1)
            if attempt == retries:
                logger.error(f"Failed to generate text after {retries} attempts.")
                return f"Error: Failed to generate text after {retries} attempts: {str(e)}"
    return "Error: No response from LLM after retries."

def get_mission_data(query: str) -> str:
    """Fetches mission data with specific launch date/time for lunar missions."""
    logger.info(f"Fetching mission data for query: {query}")
    query_lower = query.lower()

    # Regex for flexible query matching
    mars_launch_pattern = re.compile(r'(optimal.*launch.*(?:window|date|time)|launch.*window).*mars', re.IGNORECASE)
    lunar_launch_pattern = re.compile(r'(optimal.*launch.*(?:window|date|time)|launch.*window).*moon', re.IGNORECASE)
    orion_pattern = re.compile(r'orion.*(?:spacecraft|capability)', re.IGNORECASE)
    radiation_pattern = re.compile(r'radiation.*exposure', re.IGNORECASE)
    trajectory_pattern = re.compile(r'mars.*trajectory|moon.*trajectory', re.IGNORECASE)

    # Use current date as reference for lunar mission launch
    current_date = datetime(2025, 9, 3, 6, 43)  # Current date: September 3, 2025, 06:43 AM EDT
    launch_date = current_date.strftime("%B %d, %Y, %I:%M %p EDT")

    if lunar_launch_pattern.search(query) or query_lower == "optimal launch windows for earth to moon mission with orion spacecraft, considering radiation exposure in van allen belts and crew safety constraints":
        return (f"Optimal launch window scheduled for {launch_date}. Daily windows are available, with timing optimized for landing site illumination and trajectory. "
                "For Orion, launches minimize time in the Van Allen belts (~90 minutes transit). "
                "Radiation exposure is mitigated by high-speed transit and spacecraft shielding (~0.3 mSv/day in deep space). "
                "Crew safety is ensured via storm shelter, real-time monitoring, and abort options.")
    elif mars_launch_pattern.search(query) or query_lower == "optimal launch windows for mars":
        return ("Optimal launch windows for Mars occur every 26 months (approximately 780 days) due to the synodic period, aligning Earth and Mars for a Hohmann transfer orbit. "
                "Upcoming windows: October 2026 (solar minimum, low galactic cosmic ray exposure), December 2028, February 2031. "
                "Radiation exposure during transit is minimized during solar minimum (e.g., 2025-2026 for Solar Cycle 25, ~0.3 mSv/day with Orion’s shielding). "
                "Crew safety is ensured via Orion’s storm shelter, real-time radiation monitoring, and optimized trajectory planning.")
    elif orion_pattern.search(query):
        return ("Orion spacecraft: Crew capacity of 4-6, AJ10 engine (specific impulse ~319 s), "
                "radiation shielding for deep space, life support for 21 days, delta-v capability ~1.4 km/s.")
    elif radiation_pattern.search(query):
        return ("Radiation exposure can be minimized by launching during solar minimum (e.g., 2025-2026 for Cycle 25). "
                "Orion’s shielding reduces exposure to ~0.3 mSv/day in deep space.")
    elif trajectory_pattern.search(query):
        return ("Moon trajectories use direct or low-energy transfers, requiring ~3-5 days travel time. "
                "Delta-v for Earth escape: ~3.2 km/s; lunar orbit insertion: ~0.8 km/s.")
    else:
        logger.warning(f"No specific data found for query: '{query}'")
        return f"No specific data found for query: '{query}'."

# --- SQLite Database Functions ---
DATABASE_NAME = 'mission_plan.db'

def setup_database():
    """Creates SQLite tables for mission plans and agent decisions."""
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS plan_sections (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                section_name TEXT NOT NULL UNIQUE,
                data TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS agent_decisions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                thought TEXT,
                tool TEXT,
                tool_input TEXT,
                timestamp TEXT
            )
        ''')
        conn.commit()
        logger.info("Database initialized successfully.")
    except sqlite3.Error as e:
        logger.error(f"Database setup error: {str(e)}")
        raise
    finally:
        conn.close()

def populate_database():
    """Populates the database with initial mission plan data before planning."""
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()
        # Use current date as reference for lunar mission launch
        current_date = datetime(2025, 9, 3, 8, 0)  # Set launch to 08:00 AM EDT on Sep 3, 2025
        launch_date = current_date.strftime("%B %d, %Y, %I:%M %p EDT")
        initial_sections = {
            "Launch Date and Time": (
                f"Launch scheduled for {launch_date}. Daily windows are available, with timing optimized for landing site illumination and trajectory. "
                "For Orion, launches are planned to minimize time in the Van Allen belts (~90 minutes transit). "
                "Radiation exposure is mitigated by high-speed transit and spacecraft shielding (~0.3 mSv/day in deep space). "
                "Crew safety is ensured via storm shelter, real-time monitoring, and abort options."
            ),
            "Trajectory": (
                "Moon trajectories use direct or low-energy transfers, requiring ~3-5 days travel time. "
                "Delta-v for Earth escape: ~3.2 km/s; lunar orbit insertion: ~0.8 km/s."
            ),
            "Maneuver Schedule": (
                "Maneuvers include Earth escape burn (~3.2 km/s delta-v), mid-course corrections (estimated 2-3 burns, ~10-50 m/s each), and lunar orbit insertion (~0.8 km/s). "
                "Corrections will be scheduled based on real-time navigation data to ensure precise arrival."
            ),
            "Communication Plan": (
                "Communication uses S-band for telemetry and voice, Ku-band for high-data-rate video and science. "
                "Data transmission rates: S-band ~256 kbps, Ku-band ~100 Mbps. "
                "Backup: UHF for surface operations. Coordination via NASA's Deep Space Network (DSN). "
                "Emergency protocols: Autonomous operations during loss of signal, with pre-defined timelines."
            ),
            "Contingency Plans": (
                "Contingency for radiation events: Crew retreats to storm shelter in Orion's central area, using stowage bags for shielding. "
                "System failures: Switch to redundant systems (life support, propulsion). "
                "Medical emergencies: Use onboard medical kit and telemedicine; abort if life-threatening. "
                "Communication loss: Autonomous mode with pre-programmed maneuvers. "
                "Landing abort: Execute ascent burn to return to orbit; alternative sites pre-identified. "
                "Experiment malfunctions: Prioritize crew safety; use backups or manual collection."
            )
        }
        for section_name, data in initial_sections.items():
            cursor.execute('''
                INSERT OR IGNORE INTO plan_sections (section_name, data) VALUES (?, ?)
            ''', (section_name, data))
        conn.commit()
        logger.info("Database populated with all mission plan sections.")
    except sqlite3.Error as e:
        logger.error(f"Database population error: {str(e)}")
    finally:
        conn.close()

def save_to_database(section_name: str, data: str):
    """Saves a mission plan section to the database."""
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT OR REPLACE INTO plan_sections (section_name, data) VALUES (?, ?)
        ''', (section_name, data))
        conn.commit()
        logger.info(f"Saved section '{section_name}' to database.")
    except sqlite3.Error as e:
        logger.error(f"Database save error for section '{section_name}': {str(e)}")
    finally:
        conn.close()

def save_decision_to_database(thought: str, tool: str, tool_input: str):
    """Saves an agent decision to the database."""
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO agent_decisions (thought, tool, tool_input, timestamp) VALUES (?, ?, ?, ?)
        ''', (thought, tool, tool_input, datetime.now().isoformat()))
        conn.commit()
        logger.info(f"Saved decision: thought='{thought[:50]}...', tool='{tool}'")
    except sqlite3.Error as e:
        logger.error(f"Database save error for decision: {str(e)}")
    finally:
        conn.close()

def get_completed_sections() -> List[str]:
    """Retrieves completed section names from the database."""
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()
        cursor.execute('SELECT section_name FROM plan_sections')
        completed_sections = [row[0] for row in cursor.fetchall()]
        logger.info(f"Completed sections: {completed_sections}")
        return completed_sections
    except sqlite3.Error as e:
        logger.error(f"Database retrieval error: {str(e)}")
        return []
    finally:
        conn.close()

def get_full_plan_from_db() -> Dict[str, str]:
    """Retrieves all mission plan sections from the database."""
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()
        cursor.execute('SELECT section_name, data FROM plan_sections ORDER BY id')
        full_plan = dict(cursor.fetchall())
        logger.info("Retrieved full plan from database.")
        return full_plan
    except sqlite3.Error as e:
        logger.error(f"Database retrieval error: {str(e)}")
        return {}
    finally:
        conn.close()

# --- Mission Details Validation ---
def validate_mission_details(mission_details: Dict[str, Any]) -> bool:
    """Validates mission details for required fields and consistency."""
    required_fields = ['origin', 'destination', 'spacecraft', 'mission_objectives', 'constraints']
    for field in required_fields:
        if field not in mission_details or not mission_details[field]:
            logger.error(f"Missing or empty mission detail: {field}")
            return False
    if "minimize radiation exposure" in mission_details['constraints'] and "launch immediately" in mission_details['constraints']:
        logger.warning("Conflicting constraints detected.")
        return False
    logger.info("Mission details validated successfully.")
    return True

# --- Section Classification ---
def classify_section(content: str, remaining_sections: List[str]) -> str:
    """Classifies content into a mission plan section."""
    content_lower = content.lower()
    keywords = {
        'Launch Date and Time': ['launch window', 'launch date', 'solar minimum', 'hohmann', 'synodic'],
        'Trajectory': ['trajectory', 'hohmann transfer', 'delta-v', 'orbit insertion'],
        'Maneuver Schedule': ['maneuver', 'burn', 'mid-course correction', 'delta-v'],
        'Communication Plan': ['communication', 'dsn', 'deep space network', 'signal', 'comms', 'loss of signal'],
        'Contingency Plans': ['contingency', 'emergency', 'backup', 'failure', 'abort', 'scenario', 'radiation event', 'shelter', 'medical', 'procedure', 'protocol', 'risk', 'evacuation']
    }

    for section, section_keywords in keywords.items():
        if section in remaining_sections and any(keyword in content_lower for keyword in section_keywords):
            logger.info(f"Classified content as '{section}' using keywords.")
            return section

    classification_prompt = f"""
    The text describes a section of a space mission plan.
    Which section does it best describe?
    Options: {remaining_sections}
    If none fit, respond with 'Unknown'.

    Text:
    ---
    {content[:500]}...
    ---
    """
    result = generate_text(classification_prompt, temperature=0.0)
    logger.info(f"LLM classified content as '{result}'.")
    return result if result in remaining_sections else 'Unknown'

# --- Autonomous Agent ---
class StatefulSpaceAgent:
    def __init__(self, llm_model: str, mission_details: Dict[str, Any]):
        """Initializes the agent."""
        if not validate_mission_details(mission_details):
            raise ValueError("Invalid mission details.")
        self.llm_model = llm_model
        self.mission_details = mission_details
        self.memory = []
        self.tools = {
            "generate_text": generate_text,
            "get_mission_data": get_mission_data
        }
        self.current_state = "planning"
        setup_database()
        populate_database()  # Populate database before planning
        logger.info(f"Agent initialized for mission: {mission_details['origin']} to {mission_details['destination']}.")

    def _get_agent_prompt(self) -> tuple[str, str]:
        """Generates prompts for the LLM."""
        completed_sections = get_completed_sections()
        remaining_sections = [s for s in ['Launch Date and Time', 'Trajectory', 'Maneuver Schedule', 'Communication Plan', 'Contingency Plans'] if s not in completed_sections]
        system_prompt = f"""
        You are a space flight planning agent for a mission from {self.mission_details['origin']} to {self.mission_details['destination']} using {self.mission_details['spacecraft']}.
        Objectives: {self.mission_details['mission_objectives']}
        Constraints: {self.mission_details['constraints']}

        Tools:
        1. `generate_text(prompt: str)`: Generates detailed text.
        2. `get_mission_data(query: str)`: Fetches mission data.
        Use `finish` with no input to end planning.

        Ensure the 'Launch Date and Time' section includes a specific launch date and time (e.g., 'September 3, 2025, 08:00 AM EDT') aligned with mission constraints.
        Respond with a string: 'thought|tool|input'
        Example: 'Need to plan launch date|get_mission_data|Optimal launch windows for Moon'

        Completed sections: {completed_sections}
        Remaining sections: {remaining_sections}
        """
        user_prompt = "Plan the next step of the mission."
        history = "\n".join([f"Tool Output: {msg['content']}" for msg in self.memory if msg['role'] == 'tool_output'])
        return system_prompt, history + "\n\n" + user_prompt

    def run(self) -> Dict[str, str]:
        """Executes the mission planning loop."""
        print(f"Agent initiated. Planning mission: {self.mission_details['origin']} to {self.mission_details['destination']}...\n")
        logger.info("Agent started mission planning.")

        max_retries = 5
        retry_count = 0
        temperature = 0.1

        while self.current_state != "finished":
            completed_sections = get_completed_sections()
            if len(completed_sections) >= 5:
                self.current_state = "finished"
                logger.info("All sections completed. Finishing planning.")
                continue

            # Check if remaining sections can be filled from database
            remaining_sections = [s for s in ['Launch Date and Time', 'Trajectory', 'Maneuver Schedule', 'Communication Plan', 'Contingency Plans'] if s not in completed_sections]
            if not remaining_sections:
                self.current_state = "finished"
                logger.info("No remaining sections to plan.")
                continue

            try:
                system_prompt, user_prompt = self._get_agent_prompt()
                llm_response_text = self.tools['generate_text'](user_prompt, system_prompt, model=self.llm_model, temperature=temperature, max_tokens=1024)
                if llm_response_text.startswith("Error:"):
                    logger.error(f"LLM error: {llm_response_text}")
                    retry_count += 1
                    temperature = min(temperature + 0.2, 1.0)
                    if retry_count >= max_retries:
                        logger.error("Max retries reached. Falling back.")
                        print("Failed to get valid LLM response. Using fallback.")
                        thought = f"LLM failed. Using database or generating text for {remaining_sections[0]}."
                        tool = "generate_text"
                        tool_input = f"Generate a detailed plan for the '{remaining_sections[0]}' section for a mission from {self.mission_details['origin']} to {self.mission_details['destination']} using {self.mission_details['spacecraft']}."
                    else:
                        self.memory.append({"role": "tool_output", "content": "LLM error. Retrying."})
                        time.sleep(1)
                        continue
                else:
                    # Parse LLM response (thought|tool|input)
                    try:
                        thought, tool, tool_input = llm_response_text.split("|")
                        if not thought or not tool or not tool_input:
                            raise ValueError("Incomplete response format")
                    except ValueError as e:
                        logger.error(f"Invalid response: {llm_response_text[:200]}...")
                        retry_count += 1
                        temperature = min(temperature + 0.2, 1.0)
                        if retry_count >= max_retries:
                            logger.error("Max retries reached. Falling back.")
                            print("Failed to parse LLM response. Using fallback.")
                            thought = f"Invalid response. Generating text for {remaining_sections[0]}."
                            tool = "generate_text"
                            tool_input = f"Generate a detailed plan for the '{remaining_sections[0]}' section for a mission from {self.mission_details['origin']} to {self.mission_details['destination']} using {self.mission_details['spacecraft']}."
                        else:
                            self.memory.append({"role": "tool_output", "content": f"Invalid response: {llm_response_text[:200]}... Retrying."})
                            time.sleep(1)
                            continue

                    # Save decision to database
                    save_decision_to_database(thought, tool, tool_input)
                    retry_count = 0
                    temperature = 0.1

                    print(f"Agent Thought: {thought}")
                    print(f"Agent Action: Using tool '{tool}' with input: '{tool_input}'")
                    logger.info(f"Agent thought: {thought}, Action: tool='{tool}', input='{tool_input}'")

                    if tool == "finish":
                        self.current_state = "finished"
                        logger.info("Agent signaled to finish planning.")
                        continue

                    if tool in self.tools:
                        result = self.tools[tool](tool_input)
                        print(f"Tool Result: {result}\n")
                        logger.info(f"Tool '{tool}' result: {result[:100]}...")

                        if tool == "generate_text" or (tool == "get_mission_data" and result.startswith("No specific data found")):
                            section_name = classify_section(result, remaining_sections)
                            if section_name == "Launch Date and Time" and not re.search(r'\w+ \d{1,2}, \d{4}, \d{1,2}:\d{2} [AP]M \w+', result):
                                # Ensure a specific launch date/time is included
                                current_date = datetime(2025, 9, 3, 8, 0)
                                launch_date = current_date.strftime("%B %d, %Y, %I:%M %p EDT")
                                result = f"Launch scheduled for {launch_date}. {result}"
                            if section_name != "Unknown" and section_name in remaining_sections:
                                save_to_database(section_name, result)
                                self.memory.append({"role": "tool_output", "content": f"Planned '{section_name}' section."})
                            else:
                                print("Could not identify section. Skipping save.")
                                logger.warning("Failed to classify section.")
                                self.memory.append({"role": "tool_output", "content": "Failed to identify section."})
                        else:
                            self.memory.append({"role": "tool_output", "content": result})
                    else:
                        print(f"Error: Tool '{tool}' not found.")
                        logger.error(f"Tool '{tool}' not found.")
                        self.memory.append({"role": "tool_output", "content": "Tool not found"})
            except Exception as e:
                logger.error(f"Unexpected error: {str(e)}")
                print(f"Unexpected error: {e}. Falling back.")
                retry_count += 1
                if retry_count >= max_retries:
                    logger.error("Max retries reached. Aborting.")
                    self.current_state = "finished"
                    break
                thought = f"Error: {str(e)}. Generating text for {remaining_sections[0]}."
                tool = "generate_text"
                tool_input = f"Generate a detailed plan for the '{remaining_sections[0]}' section for a mission from {self.mission_details['origin']} to {self.mission_details['destination']} using {self.mission_details['spacecraft']}."
                save_decision_to_database(thought, tool, tool_input)
                print(f"Agent Thought: {thought}")
                print(f"Agent Action: Using tool '{tool}' with input: '{tool_input}'")
                result = self.tools[tool](tool_input)
                print(f"Tool Result: {result}\n")
                logger.info(f"Tool '{tool}' result: {result[:100]}...")
                section_name = classify_section(result, remaining_sections)
                if section_name == "Launch Date and Time" and not re.search(r'\w+ \d{1,2}, \d{4}, \d{1,2}:\d{2} [AP]M \w+', result):
                    # Ensure a specific launch date/time is included
                    current_date = datetime(2025, 9, 3, 8, 0)
                    launch_date = current_date.strftime("%B %d, %Y, %I:%M %p EDT")
                    result = f"Launch scheduled for {launch_date}. {result}"
                if section_name != "Unknown" and section_name in remaining_sections:
                    save_to_database(section_name, result)
                    self.memory.append({"role": "tool_output", "content": f"Planned '{section_name}' section."})
                else:
                    print("Could not identify section. Skipping save.")
                    logger.warning("Failed to classify section.")
                    self.memory.append({"role": "tool_output", "content": "Failed to identify section."})
                time.sleep(1)

        print("Mission planning complete. Final plan:")
        final_plan = get_full_plan_from_db()
        if not final_plan:
            print("Unable to complete the mission plan.")
            logger.error("Mission planning failed to produce a plan.")
        else:
            for section_name, section_content in final_plan.items():
                print(f"--- {section_name} ---")
                print(section_content)
                print("\n")
        return final_plan

# --- Example Usage ---
if __name__ == "__main__":
    # Clean up previous database
    if os.path.exists(DATABASE_NAME):
        os.remove(DATABASE_NAME)
        logger.info(f"Removed existing database: {DATABASE_NAME}")

    mission_details = {
        "origin": "Earth",
        "destination": "Moon",
        "spacecraft": "Orion spacecraft",
        "mission_objectives": ["Land on the Moon", "Conduct scientific experiments"],
        "constraints": ["Minimize radiation exposure", "Ensure crew safety"]
    }

    try:
        agent = StatefulSpaceAgent(
            llm_model="deepseek-reasoner",
            mission_details=mission_details
        )
        final_plan = agent.run()
    except Exception as e:
        print(f"Failed to run agent: {str(e)}")
        logger.error(f"Agent failed: {str(e)}")

Agent initiated. Planning mission: Earth to Moon...

Mission planning complete. Final plan:
--- Launch Date and Time ---
Launch scheduled for September 03, 2025, 08:00 AM EDT. Daily windows are available, with timing optimized for landing site illumination and trajectory. For Orion, launches are planned to minimize time in the Van Allen belts (~90 minutes transit). Radiation exposure is mitigated by high-speed transit and spacecraft shielding (~0.3 mSv/day in deep space). Crew safety is ensured via storm shelter, real-time monitoring, and abort options.


--- Trajectory ---
Moon trajectories use direct or low-energy transfers, requiring ~3-5 days travel time. Delta-v for Earth escape: ~3.2 km/s; lunar orbit insertion: ~0.8 km/s.


--- Maneuver Schedule ---
Maneuvers include Earth escape burn (~3.2 km/s delta-v), mid-course corrections (estimated 2-3 burns, ~10-50 m/s each), and lunar orbit insertion (~0.8 km/s). Corrections will be scheduled based on real-time navigation data to ensure